# Summary of Notebook
    qualified products as high-range, mid-range or low-range based on price
    qualified busiest day of week as busiest day, regularly busy or least busy based on the count for each day of week
    qualified hour of day as most orders, average orders, fewest orders based on the count for each hour of the day
    Exported rows: 30330084
    Saved as pickle
    Saved in analysis
    Saved as all_busy_day_hour.pkl

In [1]:
#importing libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import datetime as dt

In [2]:
#import orders_products_customers_combined
df_all = pd.read_pickle(r'C:\Users\Selena\Desktop\3-22 Instacart Basket Analysis\02 Data\Prepared Data\orders_products_customers_combined.pkl')
path = r'C:\Users\Selena\Desktop\3-22 Instacart Basket Analysis'
df_all = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_customers_combined.pkl'))

In [3]:
df_all.head()

,order_id,user_id,order_number,day_of_week_ordered,hour_of_day_ordered,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,price,gender,state,age,date_joined,dependents,living_situation,income
0,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,Female,Alabama,31,2019-02-17,3,married,40423
1,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,Female,Alabama,31,2019-02-17,3,married,40423
2,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,Female,Alabama,31,2019-02-17,3,married,40423
3,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,Female,Alabama,31,2019-02-17,3,married,40423
4,3367565,1,6,2,7,19.0,196,1,1,Soda,77,7,9.0,Female,Alabama,31,2019-02-17,3,married,40423


In [4]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30330084 entries, 0 to 30330083
Data columns (total 20 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   order_id                int32         
 1   user_id                 int32         
 2   order_number            int16         
 3   day_of_week_ordered     int16         
 4   hour_of_day_ordered     int16         
 5   days_since_prior_order  float64       
 6   product_id              int32         
 7   add_to_cart_order       int16         
 8   reordered               int16         
 9   product_name            category      
 10  aisle_id                int16         
 11  department_id           int16         
 12  price                   float64       
 13  gender                  category      
 14  state                   category      
 15  age                     int16         
 16  date_joined             datetime64[ns]
 17  dependents              int16         
 18  

In [5]:
#seeing if any data is missing
df_all.isnull().sum()

order_id                  0
user_id                   0
order_number              0
day_of_week_ordered       0
hour_of_day_ordered       0
days_since_prior_order    0
product_id                0
add_to_cart_order         0
reordered                 0
product_name              0
aisle_id                  0
department_id             0
price                     0
gender                    0
state                     0
age                       0
date_joined               0
dependents                0
living_situation          0
income                    0
dtype: int64

In [6]:
#only going to work with a subset of the data and look at the first million rows
df = df_all[:1000000]

In [7]:
#using .loc to create price categories
df.loc[df['price'] > 15, 'price_range_loc'] = 'High-range product'

C:\Users\Selena\anaconda3\lib\site-packages\pandas\core\indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
C:\Users\Selena\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [8]:
df.loc[(df['price'] <= 15) & (df['price'] > 5), 'price_range_loc'] = 'Mid-range product' 

In [9]:
df.loc[df['price'] <= 5, 'price_range_loc'] = 'Low-range product'

In [10]:
df['price_range_loc'].value_counts(dropna = False)

Mid-range product    631969
Low-range product    368031
Name: price_range_loc, dtype: int64

In [11]:
df_all.loc[df_all['price'] >15,'price_range_loc'] = 'High-range product'

In [12]:
df_all.loc[(df_all['price'] <=15) & (df_all['price'] >5), 'price_range_loc'] = 'Mid-range product'

In [13]:
df_all.loc[df_all['price'] <=5,'price_range_loc'] = 'Low-range product'

In [14]:
df_all['price_range_loc'].value_counts(dropna = False)

Mid-range product     20463465
Low-range product      9481550
High-range product      385069
Name: price_range_loc, dtype: int64

In [15]:
#which day of week has highest frequency of orders?
df_all['day_of_week_ordered'].value_counts(dropna = False)

0    5779313
1    5303949
6    4191133
5    3952504
2    3947737
3    3600757
4    3554691
Name: day_of_week_ordered, dtype: int64

In [16]:
#now adding meaning to the data. Defining the busy level for day of week
result = []

for value in df_all["day_of_week_ordered"]:
  if value == 0:
    result.append("Busiest day")
  elif value == 1:
    result.append("Busiest day")
  elif value == 3:
    result.append("Least busy")
  elif value == 4:
    result.append("Least busy")
  else:
    result.append("Regularly busy")

In [17]:
result

['Least busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Regularly busy',
 'Busiest day',
 'Busiest day',
 'Busiest day',
 'Least busy',
 'Busiest day',
 'Regularly busy',
 'Regularly busy',
 'Busiest day',
 'Busiest day',
 'Regularly busy',
 'Regularly busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Busiest day',
 'Busiest day',
 'Regularly busy',
 'Regularly busy',
 'Busiest day',
 'Regularly busy',
 'Regularly busy',
 'Busiest day',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Least busy',
 'Regularly busy',
 'Busiest day',
 'Busiest day',
 'Regularly busy',
 'Least busy',
 'Least busy',
 'Regularly busy',
 'Regularly busy',
 'Busiest day',
 'Busiest day',
 'Regularly busy',
 'Busiest day',
 'Busiest day',
 'Least busy',
 'Regularly busy',
 'Busiest day',
 'Busiest day',
 'Busiest day',
 'Busiest day',
 'Busiest day',
 'Regularly busy',
 'Regularly busy',
 'Busiest day',
 'Least busy',
 'Busiest day',
 'Regularly 

In [18]:
#for everything to make sense, need to add the result to the df
df_all['busiest_day'] = result

In [19]:
#looking at the frequency of the levels of categories of busy and number matches original day of week data
df_all['busiest_day'].value_counts(dropna = False)

Regularly busy    12091374
Busiest day       11083262
Least busy         7155448
Name: busiest_day, dtype: int64

In [20]:
df_all.head()

,order_id,user_id,order_number,day_of_week_ordered,hour_of_day_ordered,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,price,gender,state,age,date_joined,dependents,living_situation,income,price_range_loc,busiest_day
0,2398795,1,2,3,7,15.0,196,1,1,Soda,...,9.0,Female,Alabama,31,2019-02-17,3,married,40423,Mid-range product,Least busy
1,473747,1,3,3,12,21.0,196,1,1,Soda,...,9.0,Female,Alabama,31,2019-02-17,3,married,40423,Mid-range product,Least busy
2,2254736,1,4,4,7,29.0,196,1,1,Soda,...,9.0,Female,Alabama,31,2019-02-17,3,married,40423,Mid-range product,Least busy
3,431534,1,5,4,15,28.0,196,1,1,Soda,...,9.0,Female,Alabama,31,2019-02-17,3,married,40423,Mid-range product,Least busy
4,3367565,1,6,2,7,19.0,196,1,1,Soda,...,9.0,Female,Alabama,31,2019-02-17,3,married,40423,Mid-range product,Regularly busy


In [21]:
#which hour of day has highest frequency of orders?
df_all['hour_of_day_ordered'].value_counts(dropna = False)

10    2593848
11    2564684
14    2517325
15    2487685
13    2487618
12    2445950
16    2365060
9     2311447
17    1943933
8     1622473
18    1521010
19    1169278
20     910050
7      844700
21     746281
22     592452
23     375915
6      274831
0      203483
1      108115
5       82713
2       63966
4       49403
3       47864
Name: hour_of_day_ordered, dtype: int64

In [23]:
#now categorizing the busiest order times
result = []

for value in df_all["hour_of_day_ordered"]:
  if value == 10:
    result.append("Most Orders")
  elif value == 11:
    result.append("Most Orders")
  elif value == 14:
    result.append("Most Orders")
  elif value == 15:
    result.append("Most Orders")
  elif value == 13:
    result.append("Most Orders")
  elif value == 12:
    result.append("Most Orders")
  elif value == 5:
    result.append("Fewest Orders")
  elif value == 2:
    result.append("Fewest Orders")
  elif value == 4:
    result.append("Fewest Orders")
  elif value == 3:
    result.append("Fewest Orders")
  else:
    result.append("Average Orders")

In [24]:
result

['Average Orders',
 'Most Orders',
 'Average Orders',
 'Most Orders',
 'Average Orders',
 'Average Orders',
 'Most Orders',
 'Average Orders',
 'Average Orders',
 'Average Orders',
 'Average Orders',
 'Most Orders',
 'Average Orders',
 'Most Orders',
 'Most Orders',
 'Most Orders',
 'Average Orders',
 'Most Orders',
 'Average Orders',
 'Most Orders',
 'Average Orders',
 'Most Orders',
 'Most Orders',
 'Most Orders',
 'Most Orders',
 'Average Orders',
 'Most Orders',
 'Most Orders',
 'Average Orders',
 'Average Orders',
 'Most Orders',
 'Most Orders',
 'Most Orders',
 'Average Orders',
 'Most Orders',
 'Most Orders',
 'Most Orders',
 'Average Orders',
 'Most Orders',
 'Most Orders',
 'Most Orders',
 'Most Orders',
 'Most Orders',
 'Most Orders',
 'Most Orders',
 'Most Orders',
 'Average Orders',
 'Most Orders',
 'Average Orders',
 'Most Orders',
 'Average Orders',
 'Most Orders',
 'Most Orders',
 'Average Orders',
 'Average Orders',
 'Most Orders',
 'Most Orders',
 'Average Orders',
 'M

In [25]:
#for everything to make sense, need to add the result to the df
df_all['busiest_period_of_day'] = result

In [26]:
df_all.head()

,order_id,user_id,order_number,day_of_week_ordered,hour_of_day_ordered,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,gender,state,age,date_joined,dependents,living_situation,income,price_range_loc,busiest_day,busiest_period_of_day
0,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Female,Alabama,31,2019-02-17,3,married,40423,Mid-range product,Least busy,Average Orders
1,473747,1,3,3,12,21.0,196,1,1,Soda,...,Female,Alabama,31,2019-02-17,3,married,40423,Mid-range product,Least busy,Most Orders
2,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Female,Alabama,31,2019-02-17,3,married,40423,Mid-range product,Least busy,Average Orders
3,431534,1,5,4,15,28.0,196,1,1,Soda,...,Female,Alabama,31,2019-02-17,3,married,40423,Mid-range product,Least busy,Most Orders
4,3367565,1,6,2,7,19.0,196,1,1,Soda,...,Female,Alabama,31,2019-02-17,3,married,40423,Mid-range product,Regularly busy,Average Orders


In [27]:
#looking at the frequency of the levels of categories of busy and number matches original hour of day order placed data
df_all['busiest_period_of_day'].value_counts(dropna = False)

Most Orders       15097110
Average Orders    14989028
Fewest Orders       243946
Name: busiest_period_of_day, dtype: int64

In [28]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30330084 entries, 0 to 30330083
Data columns (total 23 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   order_id                int32         
 1   user_id                 int32         
 2   order_number            int16         
 3   day_of_week_ordered     int16         
 4   hour_of_day_ordered     int16         
 5   days_since_prior_order  float64       
 6   product_id              int32         
 7   add_to_cart_order       int16         
 8   reordered               int16         
 9   product_name            category      
 10  aisle_id                int16         
 11  department_id           int16         
 12  price                   float64       
 13  gender                  category      
 14  state                   category      
 15  age                     int16         
 16  date_joined             datetime64[ns]
 17  dependents              int16         
 18  

In [29]:
#exporting the new dataframe as a pickle
df_all.to_pickle(os.path.join(path, '04 Analysis', 'all_busy_day_hour.pkl'))